In [10]:
from tensorflow import keras
from keras import layers
import os
import numpy as np
import matplotlib

In [11]:
original_dim = (256,256)
intermediate_dim = 64
latent_dim = 2

In [12]:
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

inputs = keras.Input(shape=original_dim)

# Create/Apply Conv Layers
bn1 = layers.BatchNormalization(input_shape=(256, 256, 3))
c1 = layers.Conv2D(32, (3,3), activation='relu')(bn1)
mp1 = layers.MaxPooling2D((2, 2))(c1)
dpo1 = layers.Dropout(0.3)(mp1)

bn2 = layers.BatchNormalization()(dpo1)
c2 = layers.Conv2D(64, (3,3), activation='relu')(bn2)
mp2 = layers.MaxPooling2D((2, 2))(c2)
dpo2 = layers.Dropout(0.3)(mp2)

bn3 = layers.BatchNormalization()(dpo2)
c3 = layers.Conv2D(128, (3,3), activation='relu')(bn3)
mp3 = layers.MaxPooling2D((2, 2))(c3)
dpo3 = layers.Dropout(0.3)(mp3)

bn4 = layers.BatchNormalization()(dpo3)
c4 = layers.Conv2D(64, (3,3), activation='relu')(bn4)
mp4 = layers.MaxPooling2D((2, 2))(c4)
dpo4 = layers.Dropout(0.3)(mp4)

bn5 = layers.BatchNormalization()(dpo4)
c5 = layers.Conv2D(32, (3,3), activation='relu')(bn5)
mp5 = layers.MaxPooling2D((2, 2))(c5)
dpo5 = layers.Dropout(0.3)(mp5)

# Flatten the layer
flat = layers.Flatten()(dpo5)

h = layers.Dense(intermediate_dim, activation='relu')(flat)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)
z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'tuple'

In [ ]:
# Create the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [19]:
from PIL import Image
subreddits = os.listdir("Data")
for subreddit in subreddits:
    subreddit_folder = os.path.join("Data", subreddit)
    image_paths = [os.path.join(subreddit_folder, filename) for filename in os.listdir(subreddit_folder) if ".csv" not in filename]
    for image_path in image_paths:
        image = Image.open(image_path)
        w = image.width
        h = image.height
        crop = min(w, h)
        crop_dim = (w//2 - crop//2, h//2 - crop//2, w//2 + crop//2, h//2 + crop//2)
        image = image.crop(crop_dim)
        image.thumbnail(original_dim)
        # image.show()
        image_array = np.asarray(image)
        print(image_array.shape)
        

(256, 256, 3)
